In [ ]:
import sklearn 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from functools import reduce
import os
from dmba import classificationSummary
import warnings
warnings.filterwarnings('ignore')
#imputation 
from sklearn.impute import SimpleImputer
%matplotlib inline

In [ ]:
os.getcwd()

In [ ]:
'/Users/zhao/Desktop/Data_Mining'

In [ ]:
#not including medications
os.chdir('/Users/zhao/Desktop/Data_Mining/data')
files= ['demographic.csv', 'diet.csv', 'questionnaire.csv'] #only use demo, diet, and questionnaire 
demo= pd.read_csv('demographic.csv')
diet= pd.read_csv('diet.csv')
qr= pd.read_csv('questionnaire.csv')

In [ ]:
print('demo', demo.shape)
print('diet', diet.shape)
print('ques', qr.shape)

In [ ]:
demo (10175, 47)
diet (9813, 168)
ques (10175, 953)

In [ ]:
ls=[]
for file in files: 
    df= pd.read_csv(file)
    ls.append(df)
df_merge= reduce(lambda x,y: pd.merge(x, y,  how='inner', on= 'SEQN', suffixes=('', '_drop')), ls)
df_merge.drop([col for col in df_merge.columns if 'drop' in col], axis=1, inplace=True)
print("merged df shape:", df_merge.shape)

In [ ]:
merged df shape: (9813, 1166)

In [ ]:
#check for duplicated SEQN
df_merge.SEQN.duplicated().value_counts()

In [ ]:
False    9813
Name: SEQN, dtype: int64

In [ ]:
cbook= pd.read_csv('nhanes_2013_2014_codebook.csv')
#convert variable names to upper class to match with df_merge
cbook['variable']= cbook['variable'].str.upper()

In [ ]:
#replace all 7s and 9s as null 
df_merge.replace({7:None, 9:None, 77:None,99:None,777:None,999:None,7777:None,9999:None,77777:None,99999:None,
            777777:None,999999:None,55:None,555:None,5555:None,8:None,88:None}, inplace=True)

In [ ]:
#test
df_merge.DPQ050.describe

In [ ]:
<bound method NDFrame.describe of 0       0.0
1       0.0
2       0.0
3       NaN
4       3.0
       ... 
9808    0.0
9809    NaN
9810    NaN
9811    NaN
9812    NaN
Name: DPQ050, Length: 9813, dtype: float64>

In [ ]:
#create phq9 scores and drop rows with any item missing 
df_merge['phq9']= df_merge[['DPQ010','DPQ020','DPQ030','DPQ040', 'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090']].sum(axis=1, skipna=False)

In [ ]:
#To prep for visualization, assign categories to the original data and data after remving missing phq9 scores
orig = df_merge #original data set to be used for comparison
orig['DataSet'] = np.where(df_merge['phq9'].notna(), 'Cleaned Data', 'Original Data')  #add column

In [ ]:
#drop rows with any item missing for phq9 

df_merge= df_merge[df_merge['phq9'].notna()]
print(df_merge.shape)

In [ ]:
(5372, 1168)

In [ ]:
df_merge.phq9.describe()

In [ ]:
count    5372.000000
mean        3.311616
std         4.396482
min         0.000000
25%         0.000000
50%         2.000000
75%         5.000000
max        27.000000
Name: phq9, dtype: float64

In [ ]:
font1 = {'family':'calibri','color':'darkgray','size':18}
font2 = {'family':'calibri','color':'black','size':22}

plt.figure(figsize=(25,25))
ct = pd.crosstab(orig.RIDRETH3, orig.DataSet)
    
ax = ct.plot(kind='bar', stacked=True, rot= 45)
ax.legend(ncol=1, shadow=True, loc='center left', bbox_to_anchor=(1, 0.5)) 


plt.xlabel("Ethnicity", fontdict=font1)
plt.ylabel("Count of Records", fontdict = font1)
plt.title("Figure 5. Data Set Comparison by Ethnicity", fontdict = font2)
ax.set_xticklabels(['Mexican American', 'Other Hispanic', 'non-H White', 'non-H Black', 'non-H Asian'])


plt.show()

In [ ]:
<Figure size 1800x1800 with 0 Axes>

In [ ]:
findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.
findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.

In [ ]:
#drop columns with more than no missing data
df_merge= df_merge.dropna(thresh= 0.8*len(df_merge), axis=1)

In [ ]:
# exclude non-numeric values
df_merge = df_merge.select_dtypes(['number'])
print(len(df_merge.columns), 'columns left')

In [ ]:
291 columns left

In [ ]:
df_merge.info()

In [ ]:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5372 entries, 0 to 9808
Columns: 291 entries, SEQN to phq9
dtypes: float64(278), int64(13)
memory usage: 12.0 MB

In [ ]:
imp_mode=SimpleImputer(strategy='most_frequent')
df_merge = pd.DataFrame(imp_mode.fit_transform(df_merge), columns=df_merge.columns)
#could try other imputation method KNN

In [ ]:
#go over the dictionary manually and drop the unwanted variables
col = df_merge.columns
df_col = cbook[cbook['variable'].isin(col)]
df_col.to_csv(r'df_col.csv') #done in excel

In [ ]:
#remove variables that are related to administration/operation (e.g. language of interview, interpreter code)
#admin = pd.read_csv('C:/Users/AmroHassan/Google Drive/Classroom/31008 1-Data Mining Principles/Project/Depression-risk-main/Project Data/Depression-risk-main/data/administrative_col.csv')
admin = pd.read_csv('administrative_col.csv')
admin_col = admin.columns
df_merge = df_merge.drop(admin.columns, 1)

In [ ]:
#remove variables that are collected and calculated after very detailed Dietary Interviews
#(e.g. total dietary fiber)
#Can keep variables that can be easily answered
#(e.g. Total bottled water drank yesterday)
#dietary = pd.read_csv('C:/Users/AmroHassan/Google Drive/Classroom/31008 1-Data Mining Principles/Project/Depression-risk-main/Project Data/Depression-risk-main/data/diet_col.csv')
dietary = pd.read_csv('diet_col.csv')
diet_col = dietary.columns
df_merge = df_merge.drop(dietary.columns, 1)

In [ ]:
# remove one of HUQ010 and HSD010 (identical question asked in different surveys, highly correlated)
df_merge = df_merge.drop('HUQ010', 1)

In [ ]:
df_merge.info()

In [ ]:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5372 entries, 0 to 5371
Columns: 211 entries, SEQN to phq9
dtypes: float64(211)
memory usage: 8.6 MB

In [ ]:
df_merge.to_csv(r'df_merge.csv')

In [ ]:
df_merge.shape

In [ ]:
(5372, 211)

In [ ]:
sns.histplot(df_merge.phq9, discrete=True)
plt.xlabel("Numeric PHQ9 Score", fontdict=font1)
plt.ylabel("Count of Records", fontdict = font1)
plt.title("Figure 2. Distribution of PHQ9 Score", fontdict = font2)

In [ ]:
Text(0.5, 1.0, 'Figure 2. Distribution of PHQ9 Score')

In [ ]:
df_merge['mild_dep']= np.where(df_merge['phq9']>=8,1,0)
print('mild', df_merge['mild_dep'].value_counts())

In [ ]:
mild 0    4624
1     748
Name: mild_dep, dtype: int64

In [ ]:
dx = sns.countplot(x='mild_dep', data = df_merge, palette = "GnBu")
dx.set_ylabel("Count of Records", fontdict = font1)
dx.set_xlabel("PHQ9 Category", fontdict=font1)
dx.set_title('Figure 3. Depression Count (phq9>=8)', y=1.03, fontdict=font2)
dx.set_xticklabels(['Non- Dep','Dep'])

plt.show()

In [ ]:
df_merge['mod_dep']= np.where(df_merge['phq9']>=15,1,0)
print('modereate', df_merge['mod_dep'].value_counts())

In [ ]:
modereate 0    5177
1     195
Name: mod_dep, dtype: int64

In [ ]:
dx = sns.countplot(x='mod_dep', data = df_merge, palette = "GnBu")
dx.set_ylabel("Count of Records", fontdict = font1)
dx.set_xlabel("PHQ9 Category", fontdict=font1)
dx.set_title('Mod Depression Count', y=1.03, fontdict=font2)
dx.set_xticklabels(['Non-Mod Dep','Mod Dep'])

In [ ]:
[Text(0, 0, 'Non-Mod Dep'), Text(1, 0, 'Mod Dep')]

In [ ]:
df_merge['sev_dep']= np.where(df_merge['phq9']>=20,1,0)
print('severe', df_merge['sev_dep'].value_counts())

In [ ]:
severe 0    5315
1      57
Name: sev_dep, dtype: int64

In [ ]:
dx = sns.countplot(x='sev_dep', data = df_merge, palette = "GnBu")
dx.set_ylabel("Count of Records", fontdict = font1)
dx.set_xlabel("PHQ9 Category", fontdict=font1)
dx.set_title('Severe Depression Count', y=1.03, fontdict=font2)
dx.set_xticklabels(['Non-Sev Dep','Sev Dep'])

plt.show()

In [ ]:
df_merge['suicidal']= np.where(df_merge['DPQ090']>0,1,0)
print('suicidal', df_merge['suicidal'].value_counts())

In [ ]:
suicidal 0    5189
1     183
Name: suicidal, dtype: int64

In [ ]:
dx = sns.countplot(x='suicidal', data = df_merge, palette = "GnBu")
dx.set_ylabel("Count of Records", fontdict = font1)
dx.set_xlabel("PHQ9 Category", fontdict=font1)
dx.set_title('Suicidal Depression Count', y=1.03, fontdict=font2)
dx.set_xticklabels(['Non-Suicidal','Suicidal'])

plt.show()

In [ ]:
dx = sns.countplot(x='RIDRETH3', data = df_merge, palette = "GnBu")
dx.set_ylabel("Count of Records", fontdict = font1)
dx.set_xlabel("Ethnicity", fontdict=font1)
dx.set_title('Records by Ethnicity', y=1.03, fontdict=font2)
dx.set_xticklabels(['Mexican American', 'Other Hispanic', 'non-H White', 'non-H Black', 'non-H Asian'])

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(9, 9))
font1 = {'family':'calibri','color':'gray','size':10}
font2 = {'family':'calibri','color':'black','size':15}

d1=sns.histplot(data=df_merge, x="RIDAGEYR", kde=True, color="skyblue", ax=axs[0, 0])
d2=sns.countplot(data=df_merge, x="RIAGENDR", palette="GnBu", ax=axs[0, 1])
d3=sns.countplot(data=df_merge, x="DMDBORN4", palette='GnBu', ax=axs[1, 0])
d4=sns.countplot(data=df_merge, x="DMDEDUC2", palette="GnBu", ax=axs[1, 1])

d1.set_ylabel("Count of Records", fontdict = font1)
d1.set_xlabel("Age in Years", fontdict=font1)
d1.set_title('Age', y=1.03, fontdict=font2)


d2.set_ylabel("Count of Records", fontdict = font1)
d2.set_xlabel("Gender", fontdict=font1)
d2.set_title('Gender', y=1.03, fontdict=font2)
d2.set_xticklabels(['Male (1)','Female (2)'])

d3.set_ylabel("Count of Records", fontdict = font1)
d3.set_xlabel("Country of Birth", fontdict=font1)
d3.set_title('Country of Birth', y=1.03, fontdict=font2)
d3.set_xticklabels(['United States (1)','Not in United States (2)'])

d4.set_ylabel("Count of Records", fontdict = font1)
d4.set_xlabel("Level", fontdict=font1)
d4.set_title('Education Attainment', y=1.03, fontdict=font2)
d4.set_xticklabels(['less than high school', 'some high school', 'high school graduate/GED or equivalent',  
                    'some college or AA degree)', 'college graduate or above'], rotation = 90)

plt.show()

In [ ]:
findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.
findfont: Font family ['calibri'] not found. Falling back to DejaVu Sans.

In [ ]:
from sklearn.preprocessing import StandardScaler 
def normalize_data(X): 
    X_std= pd.DataFrame(StandardScaler().fit_transform(X))
    X_std.columns= X.columns
    return X_std

In [ ]:
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
#partition into X and y
target=['mild_dep']
drop_features= ['phq9',
                'SEQN',
                'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 'DPQ050', 'DPQ060','DPQ070','DPQ080', 'DPQ090',
               'mild_dep', 'sev_dep', 'suicidal', 'mod_dep']
X = df_merge[df_merge.columns.drop(drop_features)]
y= df_merge[target]
#normalize predictors
X= normalize_data(X)
# training and validation sets split
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
print('Training_X: ', train_X.shape)
print('Validation_X: ', valid_X.shape)
print('Training_y: ', train_y.shape)
print('Validation_y: ', valid_y.shape)

In [ ]:
Training_X:  (3223, 200)
Validation_X:  (2149, 200)
Training_y:  (3223, 1)
Validation_y:  (2149, 1)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
xgboost = XGBClassifier()
xgboost.fit(train_X, train_y)
importance= xgboost.feature_importances_
columns= df_merge.columns
df = pd.DataFrame({'feature': train_X.columns, 'importance': importance})
df = df.sort_values('importance', ascending=False)

In [ ]:
[21:49:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

In [ ]:
#select top 30 features 
df30= df[:30]
df30.columns=['variable', 'importance']
df30['variable'] = df30['variable'].apply(lambda x: x.upper())
df30 = pd.merge(left=df30, right=cbook, left_on='variable', right_on='variable', how='left')

In [ ]:
df30

In [ ]:
#confusion matrix
classificationSummary(valid_y, xgboost.predict(valid_X))

In [ ]:
Confusion Matrix (Accuracy 0.8809)

       Prediction
Actual    0    1
     0 1803   47
     1  209   90

In [ ]:
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
print(imblearn.__version__)

In [ ]:
0.8.1

In [ ]:
#recreate X,y for mild depression
target=['mild_dep']
drop_features= ['phq9',
                'SEQN',
                'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 'DPQ050', 'DPQ060','DPQ070','DPQ080', 'DPQ090',
               'mild_dep', 'sev_dep', 'suicidal', 'mod_dep']
X = df_merge[df_merge.columns.drop(drop_features)]
y= df_merge[target]
#normalize predictors
X= normalize_data(X)
# training and validation sets split
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
print('Training_X: ', train_X.shape)
print('Validation_X: ', valid_X.shape)
print('Training_y: ', train_y.shape)
print('Validation_y: ', valid_y.shape)

In [ ]:
Training_X:  (3223, 200)
Validation_X:  (2149, 200)
Training_y:  (3223, 1)
Validation_y:  (2149, 1)

In [ ]:
train_y.value_counts()

In [ ]:
mild_dep
0           2774
1            449
dtype: int64

In [ ]:
valid_y.value_counts()

In [ ]:
mild_dep
0           1850
1            299
dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
#oversample the minority class to have 40% the number of examples of the majority class
over = SMOTE(sampling_strategy=0.4)
#reduce the number of examples in the majority class to have 50% more than the minority class
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [ ]:
train_X, train_y=pipeline.fit_resample(train_X, train_y)

In [ ]:
train_y.value_counts()

In [ ]:
mild_dep
0           2218
1           1109
dtype: int64

In [ ]:
#test set remain the same
valid_y.value_counts()

In [ ]:
mild_dep
0           1850
1            299
dtype: int64

In [ ]:
xgboost.fit(train_X, train_y)
importance= xgboost.feature_importances_
columns= df_merge.columns
df = pd.DataFrame({'feature': train_X.columns, 'importance': importance})
df = df.sort_values('importance', ascending=False)
#select top 20 features 
df20= df[:20]
df20.columns=['variable', 'importance']
df20['variable'] = df20['variable'].apply(lambda x: x.upper())
df20 = pd.merge(left=df20, right=cbook, left_on='variable', right_on='variable', how='left')

In [ ]:
[21:50:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

In [ ]:
df20

In [ ]:
df20.to_csv(r'df20.csv')

In [ ]:
classificationSummary(valid_y, xgboost.predict(valid_X))

In [ ]:
Confusion Matrix (Accuracy 0.8809)

       Prediction
Actual    0    1
     0 1775   75
     1  181  118

In [ ]:
var_list= df20.variable.tolist()
var_list.append('mild_dep')
print(var_list)

In [ ]:
['SLQ050', 'PFQ049', 'DLQ040', 'RIAGENDR', 'HSD010', 'DBQ700', 'CBQ550', 'DUQ370', 'SMQ863', 'OHQ770', 'MCQ010', 'DIQ160', 'MCQ080', 'INQ080', 'MCQ160F', 'HSQ510', 'PAQ665', 'DIQ050', 'MCQ160K', 'WHQ040', 'mild_dep']

In [ ]:
#final dfs of balanced training data 
train_y= train_y.filter(var_list)
train_X= train_X.filter(var_list)
#final testing set 
valid_y= valid_y.filter(var_list)
valid_X= valid_X.filter(var_list)

In [ ]:
train_y.value_counts()

In [ ]:
mild_dep
0           2218
1           1109
dtype: int64

In [ ]:
valid_y.value_counts()

In [ ]:
mild_dep
0           1850
1            299
dtype: int64

In [ ]:
ax=plt.subplots(figsize=(6,6))
corr=train_X.corr()
sns.heatmap(corr, cmap=sns.cm.rocket_r)

In [ ]:
<AxesSubplot:>

In [ ]:
#get highly correlated pairs, ranked by absolute correlation values 
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

In [ ]:
get_top_abs_correlations(train_X, 10)

In [ ]:
PFQ049  DLQ040     0.414039
HSD010  DBQ700     0.375639
PFQ049  HSD010     0.346829
SLQ050  PFQ049     0.299231
DLQ040  HSD010     0.254703
SLQ050  DLQ040     0.249765
        HSD010     0.233957
HSD010  MCQ080     0.232423
MCQ010  MCQ160K    0.232217
HSD010  PAQ665     0.228167
dtype: float64

In [ ]:
train_X.shape

In [ ]:
(3327, 20)

In [ ]:
from sklearn.linear_model import LogisticRegression
logit=LogisticRegression()
logit.fit(train_X, train_y)

In [ ]:
LogisticRegression()

In [ ]:
classificationSummary(train_y, logit.predict(train_X))

In [ ]:
Confusion Matrix (Accuracy 0.7809)

       Prediction
Actual    0    1
     0 1998  220
     1  509  600

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean
from numpy import std
#cross-validation 
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(logit, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.778 (0.017)

In [ ]:
coef= logit.coef_
coef= coef.flatten()
columns= train_X.columns
df = pd.DataFrame({'feature': columns, 
                   'coefficients': coef})
df = df.sort_values('coefficients', ascending=False)
df.columns=['variable', 'coefficients']
df['variable'] = df['variable'].apply(lambda x: x.upper())
df = pd.merge(left=df, right=cbook, left_on='variable', right_on='variable', how='left')

In [ ]:
df

In [ ]:
import statsmodels.api as sm
logitsm= sm.Logit(train_y, train_X).fit()
logitsm.summary()

In [ ]:
Optimization terminated successfully.
         Current function value: 0.578656
         Iterations 6

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import RocCurveDisplay
from scipy import interp

In [ ]:
fig2 = plt.figure(figsize=[6,6])
clf= logit
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of logistic regression with 10-fold cross validation')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV 

param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': list(range(2,10)), 
    'min_samples_split': list(range(2,6))
}

gridSearch = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, 
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print('initial score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

dtree = gridSearch.best_estimator_

In [ ]:
initial score:  0.8491760933866196
parameters:  {'criterion': 'entropy', 'max_depth': 9, 'min_samples_split': 4, 'splitter': 'best'}

In [ ]:
#cross-validation using training set
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(dtree, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.857 (0.017)

In [ ]:
fig2 = plt.figure(figsize=[6,6])
clf= dtree
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of decision tree classifier with 10-fold cross validation')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV 

param_grid = {
    'max_leaf_nodes': list(range(2, 100)),
    'min_samples_split': [3,4,5,7]
}
gridSearch = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, 
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print('initial score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

rf = gridSearch.best_estimator_

In [ ]:
initial score:  0.8708166813429971
parameters:  {'max_leaf_nodes': 98, 'min_samples_split': 5}

In [ ]:
#cross-validation using training set
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(rf, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.885 (0.009)

In [ ]:
fig2 = plt.figure(figsize=[6,6])
clf= rf
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('random forest classification')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

param_grid = {
    'n_estimators': [40, 50, 60],
    'learning_rate': [1, 1.2, 1.5, 1.9]
}
gridSearch = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid, cv=5, 
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print('initial score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

ada = gridSearch.best_estimator_

In [ ]:
initial score:  0.8747381968434599
parameters:  {'learning_rate': 1.5, 'n_estimators': 60}

In [ ]:
#cross-validation using training set
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(ada, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.889 (0.012)

In [ ]:
fig3 = plt.figure(figsize=[6,6])
clf= ada
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('adaptive boosting')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    'n_estimators': [60, 70, 80],
    'max_depth': [ 7,8,9], 
    'min_impurity_decrease': [0.1, 0.2, 0.3]
}
gridSearch = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=5, 
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print('initial score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

gradient = gridSearch.best_estimator_

In [ ]:
initial score:  0.8726202894623947
parameters:  {'max_depth': 7, 'min_impurity_decrease': 0.1, 'n_estimators': 60}

In [ ]:
#cross-validation using training set
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(gradient, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.887 (0.011)

In [ ]:
fig4 = plt.figure(figsize=[6,6])
clf= gradient
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('gradient boosting')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
param_grid = {
    'n_estimators': [60, 70, 80],
    'max_depth': [ 7,8,9], 
    'min_impurity_decrease': [0.1, 0.2, 0.3]
}
gridSearch = GridSearchCV(XGBClassifier(random_state=42), param_grid, cv=5, 
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print('initial score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

xgboost = gridSearch.best_estimator_

In [ ]:
[22:00:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:00:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
initial score:  0.8636094741357899
parameters:  {'max_depth': 7, 'min_impurity_decrease': 0.1, 'n_estimators': 80}

In [ ]:
#cross-validation using training set
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# evaluate model
scores = cross_val_score(xgboost, train_X,train_y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
Accuracy: 0.883 (0.014)

In [ ]:
fig5 = plt.figure(figsize=[6,6])
clf= xgboost
tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in cv.split(train_X,train_y):
    prediction = clf.fit(train_X.iloc[train],train_y.iloc[train]).predict_proba(train_X.iloc[test])
    fpr, tpr, t = roc_curve(train_y.iloc[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.3f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('xgboost')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [ ]:
[22:01:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:49] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:01:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:576: 
Parameters: { "min_impurity_decrease" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

In [ ]:
valid_y.value_counts()

In [ ]:
mild_dep
0           1850
1            299
dtype: int64

In [ ]:
valid_X.shape

In [ ]:
(2149, 20)

In [ ]:
#retrain the selected model with full training set 
rf= sklearn.base.clone(rf)
rf.fit(train_X, train_y)
#test in the hold-out test set
classificationSummary(valid_y, rf.predict(valid_X))

In [ ]:
Confusion Matrix (Accuracy 0.8818)

       Prediction
Actual    0    1
     0 1769   81
     1  173  126

In [ ]:
from sklearn import metrics
y_pred_proba = rf.predict_proba(valid_X)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(valid_y, y_pred_proba)

#print AUC score
print(auc)

In [ ]:
0.8211289885202928

In [ ]:
from sklearn.metrics import plot_roc_curve

plot_roc_curve(rf, valid_X, valid_y)

In [ ]:
<sklearn.metrics._plot.roc_curve.RocCurveDisplay at 0x7fba004ce520>

In [ ]:
def plot_feature_importance(importance,names,model_type):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
#add codebook names 
importance= rf.feature_importances_
columns= train_X.columns
df = pd.DataFrame({'feature': columns, 
                   'importance': importance})
df = df.sort_values('importance', ascending=False)
df.columns=['variable', 'importance']
df['variable'] = df['variable'].apply(lambda x: x.upper())
df = pd.merge(left=df, right=cbook, left_on='variable', right_on='variable', how='left')

In [ ]:
plot_feature_importance(rf.feature_importances_,df.label,'RANDOM FOREST ')